In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from ces_fun_util import *
#from fun_utils_eq import *

In [2]:
## parameter values
alpha = 0.85           # labor share parameter in manufacturing
#alpha = 0.2
theta = 4             # scopevec for comparative advantage
sigma = 1.2      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1.2  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
#beta = 2.274853
#gamma= 0.784877595
beta=1.892412
gamma=0.807998928
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [3]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [4]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [5]:
class taxrow_eq:
    def __init__(self, pe, te, tb_mat, df, tax_scenario, varphi, ParaList):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.ParaList = ParaList
        self.prop = 1
        self.te = te
        self.pe = pe
        self.tb_mat = tb_mat

    ## computes equilibrium price and taxes. Also computes other equilibrium values (ie consumption, production, value of exports/imports)
    ## and stores them in self. 
    def opt_tax(self):
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi
        self.region = df['region_scenario']
        pes = np.append([self.pe], np.arange(0.1, 2.5, 0.2))
        tbs = np.append([self.tb_mat[0]], np.arange(0,1.5,0.2))
        props = np.append([self.tb_mat[1]], np.arange(0,1.1,0.5))
        self.conv = 1
        if (tax_scenario['tax_sce'] == 'global'):
            tbs = [0]
        
        if tax_scenario['tax_sce'] != 'purete' and tax_scenario['tax_sce'] != 'EP_hybrid':
            for prop in props:
                for tb in tbs:
                    for pe in pes:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, 1)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            #print(res.fun, df['region_scenario'], tax_scenario['tax_sce'], varphi)
            
        pes = np.append([self.pe], np.arange(0.1, 2, 0.5))
        props = np.append([self.tb_mat[1]], np.arange(0,2,0.5))
        if tax_scenario['tax_sce'] == 'EP_hybrid':
            for pe in pes:
                for tb in tbs:
                    for prop in props:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, np.inf)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            
            
        ## not implemented yet
        if tax_scenario['tax_sce'] == 'Baseline':
            self.tb = 0
            self.prop = 1
            self.te = 0
        
        elif tax_scenario['tax_sce'] == 'global':
            self.tb = 0
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'PC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = self.tb

        elif tax_scenario['tax_sce'] == 'EPC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'EP_hybrid':
            self.tb = opt_val[1]
            self.te = opt_val[2]
            self.prop = self.te #- self.tb

        elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'puretc' or tax_scenario[
            'tax_sce'] == 'puretp' or tax_scenario['tax_sce'] == 'EC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]

            if tax_scenario['tax_sce'] == 'puretc' or tax_scenario['tax_sce'] == 'puretp':
                self.te = self.tb

            elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'EC_hybrid':
                self.te = self.varphi

        elif tax_scenario['tax_sce'] == 'purete':
            res = minimize(self.minuswelfare, [self.pe, self.te], bounds=[(0.001, np.inf), (0, np.inf)], method='nelder-mead', args=(0),
                           tol=0.000001, options={'maxfev': 100000})
            
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            #print(res.fun)
            self.te = res.x[1]
            tb_mat = [0, 1]
            self.tb = tb_mat[0]
            self.prop = tb_mat[1]
        #print(tax_scenario['tax_sce'], res)

    def minuswelfare(self, p, te):
        pe = p[0]
        tb_mat = p[1:]
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        ParaList = self.ParaList
        df = self.df

        if tax_scenario['tax_sce'] == 'purete':
            te = p[1]
            tb_mat = [0, 1]

        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, ParaList, df)

        return obj

    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, Paralist, df):

        alpha, theta, sigma, sigmastar, epsilonD, epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        jvals = (j0_hat, j0_prime, jxbar_hat, jxbar_prime, jmbar_hat, jmbar_prime)

        # if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        Qe_prime, Qestar_prime = compute_qe(tax_scenario, petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat = comp_ce(
            pe, tb_mat, jvals, ParaList, df, tax_scenario)

        Ge_prime = Cey_prime + Cex_prime
        Gestar_prime = CeFF_prime + Cem_prime
        Ce_prime = Cey_prime + Cem_prime
        Cestar_prime = CeFF_prime + Cex_prime
        Qeworld_prime = Qe_prime + Qestar_prime

        consvals = (Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, 
                    Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat,
                    Ce_prime, Cestar_prime)

        Vgy, Vgystar, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vgx, Vgm = comp_vg(pe, tb_mat, jvals, consvals,
                                                                                           df, tax_scenario, ParaList)

        pai_g = Vgx - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha)
        subsidy_ratio = 1- ((1-jxbar_prime) * j0_prime / ((1-j0_prime) * jxbar_prime))**(1/theta)

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, consvals, tax_scenario)

        ## compute more Vg values
        vgvals = Vgx_prime, Vgm_prime
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, consvals, vgvals, jvals, ParaList, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, Ge_prime, Gestar_prime, consvals, ParaList, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        lgvals = Lg, Lgstar, Lg_prime, Lgstar_prime
        vgvals = Vg, Vgstar, Vg_prime, Vgstar_prime
        delta_Le, delta_Lestar, delta_U = comp_delta(lgvals, vgvals, Qeworld_prime, df, jvals, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)

        chg_extraction, chg_production, chg_consumption, chg_Qeworld = comp_chg(df, Qestar_prime, Gestar_prime,
                                                                                Cestar_prime, Qeworld_prime)

        welfare = delta_U / Vg * 100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld'])) / Vg * 100
        
        self.results = assign_val(Ge_prime, Gestar_prime, Lg_prime, Lgstar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                  Ve_prime, Vestar_prime, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vg_prime,
                                  Vgstar_prime, chg_Qeworld, chg_consumption, chg_extraction, chg_production, delta_Le, 
                                  delta_Lestar, leakage1, leakage2, leakage3,
                                  pai_g, pe, subsidy_ratio, varphi, welfare, welfare_noexternality, jvals, consvals)

        diff, diff1, diff2 = comp_diff(consvals, jvals, Ge_prime, Gestar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                       Vgx2_prime, pe, tax_scenario, tb_mat, te, varphi, ParaList, df)

        return abs(diff1) + abs(diff) + abs(diff2)

    ## retrive the pandas series object containing equilibrium values (prices, taxes, consumption etc)
    def retrive(self):
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['region_scenario'] = self.region
        ret['conv'] = self.conv
        
        return ret

In [6]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
df = df[df['region_scenario'] == 3]
df

region_scenario        Qe     Qestar    Qeworld       CeHH  \
regionbase                                                                   
OECD37 as Home                3  8.625495  23.650503  32.276001  11.293668   

                    CeHF      CeFH       CeFF         Ce     Cestar  \
regionbase                                                            
OECD37 as Home  2.487537  0.910579  17.584217  13.781205  18.494795   

                       Ge     Gestar    Ceworld    Geworld     jxbar     jmbar  
regionbase                                                                      
OECD37 as Home  12.204247  20.071754  32.276001  32.276001  0.049234  0.819498

In [7]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[2, 3, 4, 5, 6, 7, 8, 9])
#tax_scenario= pd.DataFrame({'tax_sce': ['global'], 'Base':[1]},index=[1])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [8]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    te = 0; #initial value of extraction tax for iteration
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    varphilist = [0, 0.1]
    #varphilist = np.arange (0,10,0.1)
    output=[]
    prevtb = [1,1,1,1]
    for varphi in varphilist:    
        te = 0 #initial value of extraction tax for iteration
        tb_mat = [0, 1]  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(prevtb, tb_mat, te, varphi, tax_scenario, ParaList))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        #print(prevtb)
        output.append(tax_df)
        print(varphi)
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [9]:
def mintemp(df, prevtb, tb_mat, te, varphi, tax_scenario, ParaList):
    pe = 0.5
    te = 0.5
    tb_mat = [0.5,0.5]
    #print(prevtb['region_scenario'])
    
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        #print(vals)
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = taxrow_eq(pe, te, tb_mat, df, tax_scenario, varphi, ParaList)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [10]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

19.18011175579642 -0.0
17.528675709571072 -0.0
18.75403933820948 -0.0
19.18011175579642 -0.0
18.50234226210585 -0.0
17.63171106605133 -0.0
16.878721534927973 -0.0
16.138830424696067 -0.0
14.695559470795539 -0.0
14.276068782760646 -0.0
13.131202323123697 -0.0
11.092548143519231 -0.0
10.004974664730513 -0.0
6.875607733077913 -0.0
7.5727920321915505 -0.0
4.550998505004014 -0.0
1.006426553434235 -0.0
-4.899516254228828 0.0
-2.221477485253911 0.0
-8.09504505803337 0.0
-4.250871349441372 0.0
-9.637760106144388 0.0
-7.821851463729532 0.0
-4.8837499082946945 0.0
1.280371359092202 -0.0
-4.432148851862472 0.0
-8.310563788824616 0.0
-1.2201095448006505 0.0
-4.373882913671922 0.0
-5.401898371465171 0.0
-2.8669001182232563 0.0
-2.1173498446245955 0.0
-1.029340044572761 0.0
-6.3231788525879695 0.0
-4.319574103845645 0.0
-5.100529891426126 0.0
-1.5609676823946512 0.0
0.4156891020014418 -0.0
1.8959876978284131 -0.0
5.428265131385678 -0.0
-1.9788268586143913 0.0
-3.5569628238226523 0.0
-2.2221940182377

In [11]:
output_list=[]
for i in range(2,len(tax_scenario)+1):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

tax_sce      regionbase  varphi        pe        tb      prop        te  \
0  Unilateral  OECD37 as Home     0.0  1.000000  0.000000  0.392478  0.000000   
1  Unilateral  OECD37 as Home     0.1  0.993692  0.039336  0.243603  0.100000   
0      purete  OECD37 as Home     0.0  1.000000  0.000000  1.000000  0.000000   
1      purete  OECD37 as Home     0.1  1.006558  0.000000  1.000000  0.073568   
0      puretc  OECD37 as Home     0.0  1.000000  0.000000  0.455748  0.000000   
1      puretc  OECD37 as Home     0.1  0.987335  0.045379  1.000000  0.045379   
0      puretp  OECD37 as Home     0.0  1.000000  0.000000  1.000000  0.000000   
1      puretp  OECD37 as Home     0.1  0.990943  0.036672  1.000000  0.036672   
0   EC_hybrid  OECD37 as Home     0.0  1.000000  0.000000  0.000000  0.000000   
1   EC_hybrid  OECD37 as Home     0.1  0.994869  0.038115  0.000282  0.100000   
0   EP_hybrid  OECD37 as Home     0.0  1.000000  0.000000  0.000000  0.000000   
1   EP_hybrid  OECD37 as Home     0.1  0.998277  0.030513  0.095810  0.095810   
0   PC_hybrid  OECD37 as Home     0.0  1.000000  0.000000  0.276105  0.000000   
1   PC_hybrid  OECD37 as Home     0.1  0.986569  0.046150  0.355356  0.046150   

   jxbar_prime  jmbar_prime  j0_prime  ...  chg_extraction  chg_production  \
0     0.049234     0.819498  0.049234  ...    7.293010e-07   -1.175026e-06   
1     0.049252     0.819498  0.048155  ...   -7.470881e-02    3.278148e-02   
0     0.049234     0.819498  0.049234  ...    5.353447e-07   -8.359349e-07   
1     0.049234     0.819498  0.049234  ...    7.742657e-02   -1.346886e-01   
0     0.049234     0.819498  0.049234  ...    7.716014e-07   -1.248979e-06   
1     0.049234     0.819498  0.047987  ...   -1.502416e-01    1.512438e-01   
0     0.049234     0.819498  0.049234  ...    9.779380e-07   -1.609714e-06   
1     0.048224     0.816250  0.048224  ...   -1.073425e-01    2.498011e-01   
0     0.049234     0.819498  0.049234  ...    1.269066e-06   -2.118688e-06   
1     0.049234     0.819498  0.048189  ...   -6.074768e-02    1.163632e-02   
0     0.049234     0.819498  0.049234  ...    5.576119e-07   -8.748643e-07   
1     0.048396     0.816810  0.048396  ...   -2.038455e-02    8.568445e-02   
0     0.049234     0.819498  0.049234  ...    6.865203e-07   -1.100233e-06   
1     0.048407     0.819498  0.047966  ...   -1.593666e-01    1.802396e-01   

   chg_consumption  chg_Qeworld         pai_g  subsidy_ratio   welfare  \
0    -2.048521e-07    -0.000002 -3.743888e-07       0.000000  0.000038   
1     8.506361e-02    -0.368550 -2.004980e-01       0.005903  0.094184   
0     1.075978e-07    -0.000002 -2.748207e-07       0.000000  0.000039   
1    -1.241057e-01    -0.216584 -3.981221e-02       0.000000  0.011781   
0    -2.729950e-07    -0.000002 -3.961038e-07       0.000000  0.000038   
1     2.444024e-01    -0.205039 -1.985919e-01       0.006718  0.011336   
0    -6.053885e-07    -0.000002 -5.020273e-07       0.000000  0.000042   
1     1.410119e-01    -0.146494 -1.676415e-01       0.000000  0.008132   
0    -1.074375e-06    -0.000001 -6.514785e-07       0.000000  0.000036   
1     9.824709e-02    -0.354785 -2.002315e-01       0.005623  0.019383   
0     7.172689e-08    -0.000002 -2.862517e-07       0.000000  0.000044   
1     5.472336e-03    -0.314437 -1.747720e-01       0.000000  0.017175   
0    -1.359354e-07    -0.000002 -3.524272e-07       0.000000  0.000039   
1     2.320110e-01    -0.217492 -1.986206e-01       0.002398  0.012025   

   welfare_noexternality  region_scenario  conv  
0               0.000038              3.0   1.0  
1               0.054070              3.0   1.0  
0               0.000039              3.0   1.0  
1              -0.011792              3.0   1.0  
0               0.000038              3.0   1.0  
1              -0.010981              3.0   1.0  
0               0.000042              3.0   1.0  
1              -0.007813              3.0   1.0  
0               0.000036              3.0   1.0  
1       

In [12]:
#outcome_t = Outcomes[Outcomes['regionbase'] == 'US as Home']
Outcomes[['Qe_prime', 'Qestar_prime', 'Ce_prime', 'Cestar_prime','Lg_prime','Lgstar_prime', 'Vg_prime', 'Vgstar_prime', 'delta_Le','delta_Lestar','welfare_noexternality']]

Qe_prime  Qestar_prime   Ce_prime  Cestar_prime   Lg_prime  Lgstar_prime  \
0  8.625495     23.650504  13.781204     18.494795  69.157399    113.739940   
1  8.331657     23.575794  13.327592     18.579859  69.092212    113.743315   
0  8.625495     23.650504  13.781204     18.494795  69.157399    113.739940   
1  8.331488     23.727930  13.688728     18.370689  69.143839    113.717637   
0  8.625495     23.650504  13.781204     18.494795  69.157399    113.739940   
1  8.570701     23.500261  13.331766     18.739197  69.097600    113.764442   
0  8.625495     23.650504  13.781204     18.494794  69.157399    113.739940   
1  8.586346     23.543161  13.493700     18.635807  68.759540    114.112539   
0  8.625495     23.650504  13.781204     18.494794  69.157399    113.739940   
1  8.331461     23.589755  13.328175     18.593042  69.095921    113.741601   
0  8.625495     23.650504  13.781204     18.494795  69.157399    113.739940   
1  8.331446     23.630118  13.461296     18.500267  68.815823    114.028666   
0  8.625495     23.650504  13.781204     18.494795  69.157399    113.739940   
1  8.567373     23.491136  13.331703     18.726806  69.010703    113.849209   

    Vg_prime  Vgstar_prime      delta_Le  delta_Lestar  welfare_noexternality  
0  91.874700    123.298640  2.659809e-07  7.293010e-07               0.000038  
1  91.785181    123.326394 -2.839416e-01 -7.447306e-02               0.054070  
0  91.874700    123.298640  1.952438e-07  5.353447e-07               0.000039  
1  91.856685    123.274463 -2.840998e-01  7.768032e-02              -0.011792  
0  91.874700    123.298640  2.814082e-07  7.716014e-07               0.000038  
1  91.786018    123.345795 -5.444678e-02 -1.492892e-01              -0.010981  
0  91.874700    123.298639  3.566605e-07  9.779381e-07               0.000042  
1  91.817876    123.325746 -3.897109e-02 -1.068560e-01              -0.007813  
0  91.874700    123.298639  4.628367e-07  1.269066e-06               0.000036  
1  91.785298    123.317668 -2.841244e-01 -6.059178e-02              -0.019234  
0  91.874701    123.298640  2.033648e-07  5.576119e-07               0.000044  
1  91.811581    123.299583 -2.841390e-01 -2.036698e-02              -0.017050  
0  91.874700    123.298640  2.503785e-07  6.865203e-07               0.000039  
1  91.786006    123.343302 -5.773130e-02 -1.582952e-01              -0.011648

In [13]:
#Outcomes.to_csv('../../output_ces/global0.csv'.format(theta, case), header=True)